In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)
        # print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pickle
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from torch.utils.data import DataLoader, TensorDataset, ConcatDataset
from sklearn.model_selection import train_test_split

# GPU device
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

In [3]:
# get file name
testFileName = list()
trainFileNameClass0 = list()
trainFileNameClass1 = list()
os.chdir('/kaggle/input/nycu-ml-pattern-recognition-hw-4/released/test')
testFileName = os.listdir()
os.chdir('../train/class_0')
trainFileNameClass0 = os.listdir()
os.chdir('../class_1')
trainFileNameClass1 = os.listdir()
os.chdir('../../')
# test file name
testFileName[0], trainFileNameClass0[0], trainFileNameClass1[0]

('7c08c8c0-0064-40c3-a7cd-dda2b4043347.pkl',
 '32767294-e76e-485e-91a0-c2a123498e01.pkl',
 '3cc6a03d-7585-48f4-82cc-89448620b059.pkl')

In [4]:
def readData(filepath, filenamelist):
    FileExtensionRemove = []
    with open(filepath+filenamelist[0], 'rb') as f:
        data = np.array([pickle.load(f)])
    FileExtensionRemove.append(filenamelist[0][:-4])
    for filename in filenamelist[1:]:
        with open(filepath+filename, 'rb') as f:
            data = np.concatenate((data, np.array([pickle.load(f)])), axis=0)
        FileExtensionRemove.append(filename[:-4])
    return data, FileExtensionRemove

def DataPreprocess(trainX0, trainX1=None):
    # combine train data
    if trainX1 is None:
        trainX = trainX0
    else:
        trainX = np.concatenate((trainX0, trainX1))
    # Transform to 0 ~ 1
    trainX = trainX.astype('float32')
    trainX /= 255
    # To pytorch
    trainX = torch.from_numpy(trainX)
    if trainX1 is None:
        trainY = None
    else:
        trainY = torch.concatenate((torch.zeros((trainX0.shape[0],1)), 
                                    torch.ones((trainX1.shape[0],1))))
    return trainX, trainY

In [5]:
DataSizeBatch = 25
train_loader = []
val_loader = []
for i in range(int(len(trainFileNameClass0)/DataSizeBatch)):
    print("read batch", i)
    startIndex = i * DataSizeBatch
    trainX0, _ = readData('train/class_0/', trainFileNameClass0[startIndex:startIndex + DataSizeBatch])
    trainX1, _ = readData('train/class_1/', trainFileNameClass1[startIndex:startIndex + DataSizeBatch])
    
    trainX, trainY = DataPreprocess(trainX0, trainX1)
    # Split data into train and validation sets
    trainX, valX, trainY, valY = train_test_split(trainX, trainY, test_size=0.2)
    
    # Create DataLoader for batching
    train_dataset = TensorDataset(trainX, trainY)
    val_dataset = TensorDataset(valX, valY)

    train_loader.append(DataLoader(train_dataset, batch_size=4, shuffle=True))
    val_loader.append(DataLoader(val_dataset, batch_size=4, shuffle=False))
    

# Try to merge dataloader
#trainLoader = DataLoader(ConcatDataset([dsa, dsb]))
#valLoader = DataLoader(ConcatDataset([dsa, dsb]))


read batch 0
read batch 1
read batch 2
read batch 3
read batch 4
read batch 5


In [6]:
# Feature extraction model using ResNet18
class FeatureExtractor(nn.Module):
    def __init__(self):
        super(FeatureExtractor, self).__init__()
        base_model = models.resnet18(weights='IMAGENET1K_V1')
        self.feature_extractor = nn.Sequential(
            *list(base_model.children())[:-1],
            nn.Flatten()
        )
    
    def forward(self, x):
        x = self.feature_extractor(x)
        return x

In [7]:
FEATURE_DIM = 512
# MIL model
class MILModel(nn.Module):
    def __init__(self):
        super(MILModel, self).__init__()
        self.feature_extractor = FeatureExtractor()
        self.classifier = nn.Linear(FEATURE_DIM, 1)
    
    def forward(self, x):
        batch_size = x.size(0)
        num_tiles = x.size(1)
        x = x.view(-1, 3, 128, 128)
        features = self.feature_extractor(x)
        features = features.view(batch_size, num_tiles, -1)
        aggregated_features, _ = torch.max(features, dim=1)
        logits = self.classifier(aggregated_features)
        output = torch.sigmoid(logits)
        return output

    def predict(self, x):
        with torch.no_grad():
            ypred_prob = self(x)
            ypred = ypred_prob.clone().detach()
            ypred[ypred < 0.5] = 0
            ypred[ypred >= 0.5] = 1
            return ypred_prob, ypred

In [8]:
# Set random seed for reproducibility
torch.manual_seed(0)

# Initialize the model
model = MILModel().to(device)
print(model)

# Define loss and optimizer
criterion = nn.BCELoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 181MB/s]


MILModel(
  (feature_extractor): FeatureExtractor(
    (feature_extractor): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (b

In [9]:

num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for tl in train_loader:
        for i, (inputs, targets) in enumerate(tl):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            running_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    avg_train_loss = running_loss / (len(tl) * len(train_loader))
    
    model.eval()
    val_loss = 0.0
    for vl in val_loader:
        with torch.no_grad():
            for inputs, targets in vl:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()
    avg_val_loss = val_loss / (len(vl)* len(val_loader))
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')
print('Training finished.')

Epoch [1/5], Train Loss: 0.7909, Val Loss: 0.5913
Epoch [2/5], Train Loss: 0.0620, Val Loss: 0.6652
Epoch [3/5], Train Loss: 0.0048, Val Loss: 0.6593
Epoch [4/5], Train Loss: 0.0029, Val Loss: 0.6660
Epoch [5/5], Train Loss: 0.0021, Val Loss: 0.6709
Training finished.


In [10]:
# Save model weight
torch.save(model.state_dict(), "/kaggle/working/weight-resnet18.pth")